<a href="https://colab.research.google.com/github/Mojtaba-Choopani/huggingface-llm-course-fa-notebooks/blob/main/chapter2-USING-TRANSFORMERS/Behind-the-pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div dir="rtl">

<b style="font-size: 30px;">پشت پرده پایپ‌لاین در ترنسفورمرها</b>

</div>

<div dir="rtl">

<p>
در فصل اول با تابع <code>pipeline()</code> آشنا شدیم.
</p>

</div>


In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
)

[{'label': 'POSITIVE', 'score': 0.9598047137260437},
 {'label': 'NEGATIVE', 'score': 0.9994558095932007}]

<div dir="rtl">

<p>
این پایپ‌لاین سه مرحله را در کنار هم قرار می‌دهد: پیش‌پردازش، عبور دادن ورودی‌ها از درون مدل، و پس‌پردازش.
</p>

</div>

---

<div dir="rtl">

<b style="font-size: 18px;"> پیش‌پردازش با توکنایزر</b>

<p>
مدل‌های ترنسفورمر نمی‌توانند متن خام را مستقیماً پردازش کنند، بنابراین ابتدا باید متن را به عدد تبدیل کنیم. این کار با <b>توکنایزر</b> انجام می‌شود که وظیفه‌اش:
</p>

<ul>
  <li>شکستن متن به توکن‌ها (کلمات، زیرکلمات یا نمادها)</li>
  <li>تبدیل هر توکن به عدد</li>
  <li>افزودن ورودی‌های اضافی لازم برای مدل</li>
</ul>

<p>
تمام این مراحل باید دقیقاً مطابق با آموزش اولیهٔ مدل انجام شود. برای همین، با استفاده از کلاس <code>AutoTokenizer</code> و متد <code>from_pretrained()</code>، توکنایزر مناسب مدل را از Model Hub بارگذاری می‌کنیم.
</p>

<p>
برای مدل پیش‌فرض تحلیل احساسات (<code>distilbert-base-uncased-finetuned-sst-2-english</code>)، این کد اجرا می‌شود.
</p>

</div>


In [ ]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

<div dir="rtl">

<b style="font-size: 18px;"> تبدیل ورودی به تنسور</b>

<p>
پس از گرفتن توکنایزر، می‌توانیم جمله‌ها را به آن بدهیم و یک خروجی آماده برای مدل دریافت کنیم. تنها کار باقی‌مانده این است که شناسه‌های ورودی (<code>input IDs</code>) را به <b>تنسور</b> تبدیل کنیم.
</p>

<p>
 <b>مدل‌های ترنسفورمر فقط تنسورها را به‌عنوان ورودی می‌پذیرند</b>، اما نگران فریم‌ورک (مثل PyTorch یا Flax) نباشید — کتابخانه  Transformers خودش آن را مدیریت می‌کند.
</p>

<p>
اگر با تنسورها آشنایی ندارید، آن‌ها را مثل آرایه‌های NumPy تصور کنید:
</p>

<ul>
  <li>عدد (۰D)</li>
  <li>بردار (۱D)</li>
  <li>ماتریس (۲D) یا بیشتر</li>
</ul>

<p>
برای تعیین نوع تنسور، از آرگومان <code>return_tensors</code> استفاده می‌کنیم (مثلاً <code>"pt"</code> برای PyTorch).
</p>

</div>


In [2]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


<div dir="rtl">

<b style="font-size: 18px;"> خروجی توکنایزر چیست؟</b>

<p>
خروجی توکنایزر یک <b>دایکشنری</b> است با دو کلید اصلی:
</p>

<ul>
  <li><code>input_ids</code>: شامل ردیف‌هایی از اعداد صحیح است که نمایانگر شناسه‌های توکن‌های هر جمله‌اند.</li>
  <li><code>attention_mask</code>: در این فصل، بعداً توضیح داده می‌شود.</li>
</ul>

<p>
هر ردیف از <code>input_ids</code> مربوط به یک جملهٔ ورودی است.
</p>
---
</div>


<div dir="rtl">

<b style="font-size: 18px;"> عبور دادن ورودی از مدل</b>

<p>
می‌توانیم مدل پیش‌آموزش‌دیده را درست مانند توکنایزر بارگیری کنیم.
</p>

<p>
کتابخانه  Transformers کلاسی به نام <code>AutoModel</code> را فراهم کرده که متدی به نام <code>from_pretrained()</code> دارد؛ دقیقاً مشابه توکنایزر.
</p>

</div>


In [ ]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

<div dir="rtl">

<p>
این معماری تنها شامل ماژول پایه‌ی ترنسفورمر است: با دریافت ورودی‌هایی، خروجی‌ای تولید می‌کند که آن را <b>بردارهای پنهان</b> می‌نامیم، که با نام <b>ویژگی‌ها</b> نیز شناخته می‌شوند — برداری با ابعاد بالا که نشان‌دهندهٔ درک مدل از معنای توکن در زمینهٔ جمله است.
</p>

<p>
اگرچه این بردارهای پنهان به‌تنهایی می‌توانند مفید باشند، اما معمولاً به بخش دیگری از مدل داده می‌شوند که <b>«هد» (head)</b> نام دارد.
در فصل اول دیدیم که وظایف مختلف می‌توانند با همین معماری پایه انجام شوند، اما هرکدام از این وظایف دارای <b>هد متفاوتی</b> هستند.
</p>

</div>
---

<div dir="rtl">

<b style="font-size: 18px;"> یک بردار با ابعاد بالا؟</b>

<p>
برداری که توسط ماژول ترنسفورمر تولید می‌شود معمولاً <b>بزرگ</b> است. این بردار معمولاً <b>سه بُعد</b> دارد:
</p>

<ul>
  <li><b>Batch size:</b> تعداد دنباله‌هایی که هم‌زمان پردازش می‌شوند (در مثال ما ۲ مورد).</li>
  <li><b>Sequence length:</b> طول نمایش عددی دنباله (در مثال ما ۱۶).</li>
  <li><b>Hidden size:</b> بُعد بردار مربوط به هر ورودی مدل.</li>
</ul>

</div>


In [4]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([2, 16, 768])


<div dir="rtl">

<b style="font-size: 18px;"> هدهای مدل: معنا بخشیدن به اعداد</b>

<p>
هدهای مدل، <b>بردارهای پنهان با ابعاد بالا</b> را به‌عنوان ورودی می‌گیرند و آن‌ها را به بُعدی دیگر نگاشت می‌کنند. این هدها معمولاً از یک یا چند <b>لایه‌ی خطی (Linear Layer)</b> تشکیل شده‌اند.
</p>

<p>
خروجی ماژول ترنسفورمر مستقیماً به هد مدل ارسال می‌شود تا پردازش شود.
</p>

<p>
در کتابخانه  Transformers معماری‌های متنوعی وجود دارد که هرکدام برای یک وظیفه‌ی خاص طراحی شده‌اند. فهرست زیر کامل نیست، اما چند نمونه از آن‌ها را نشان می‌دهد:
</p>

<ul>
  <li><code>Model</code> (برای دریافت بردارهای پنهان)</li>
  <li><code>ForCausalLM</code></li>
  <li><code>ForMaskedLM</code></li>
  <li><code>ForMultipleChoice</code></li>
  <li><code>ForQuestionAnswering</code></li>
  <li><code>ForSequenceClassification</code></li>
  <li><code>ForTokenClassification</code></li>
  <li>و مدل‌های دیگر </li>
</ul>

<p>
در مثال ما، به مدلی نیاز داریم که دارای <b>هد برای طبقه‌بندی دنباله</b> باشد (تا بتواند جملات را به مثبت یا منفی طبقه‌بندی کند). بنابراین، از کلاس <code>AutoModel</code> استفاده نمی‌کنیم، بلکه از <code>AutoModelForSequenceClassification</code> استفاده خواهیم کرد.
</p>

</div>


In [5]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

<div dir="rtl">

<p>
اکنون اگر به <b>شکل (shape)</b> خروجی‌های خود نگاه کنیم، خواهیم دید که بُعد آن‌ها بسیار کمتر شده است: <b>هد مدل</b>، بردارهای پُربُعدی که قبلاً دیدیم را به‌عنوان ورودی می‌گیرد و <b>بردارهایی با دو مقدار</b> (یکی برای هر برچسب) به‌عنوان خروجی تولید می‌کند.
از آن‌جایی که تنها دو جمله و دو برچسب داریم، خروجی‌ای که از مدل دریافت می‌کنیم دارای شکل ۲ × ۲ خواهد بود.
</p>

</div>


In [6]:
print(outputs.logits.shape)

torch.Size([2, 2])


<div dir="rtl">

<b style="font-size: 18px;"> پردازش پس از خروجی (Postprocessing)</b>

<p>
مقدارهایی که به‌عنوان خروجی از مدل دریافت می‌کنیم، لزوماً به‌تنهایی معنا‌دار نیستند. بیایید نگاهی به آن‌ها بیندازیم:
</p>

</div>


In [7]:
print(outputs.logits)

tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>)


<div dir="rtl">

<p>
مدل ما برای جملهٔ اول مقدارهای <code>[-1.5607, 1.6123]</code> و برای جملهٔ دوم مقدارهای <code>[4.1692, -3.3464]</code> را پیش‌بینی کرده است. این‌ها <b>احتمال نیستند</b>، بلکه <b>لاجیت</b> هستند — یعنی امتیازهای خام و نرمال‌نشده‌ای که از آخرین لایهٔ مدل خارج می‌شوند.
</p>

<p>
برای تبدیل این مقادیر به احتمال، باید آن‌ها را از یک <b>لایه‌ی SoftMax</b> عبور دهیم.
</p>

<p>
تمام مدل‌های  Transformers <b>لاجیت‌ها</b> را به‌عنوان خروجی برمی‌گردانند، زیرا در فرآیند آموزش، معمولاً <b>تابع فعال‌سازی نهایی</b> (مانند SoftMax) با <b>تابع زیان نهایی</b> (مانند cross-entropy) ترکیب می‌شود.
</p>

</div>


In [ ]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

tensor([[4.0195e-02, 9.5980e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward>)

<div dir="rtl">

<p>
اکنون می‌بینیم که مدل برای جملهٔ اول مقدارهای <code>[0.0402, 0.9598]</code> و برای جملهٔ دوم مقدارهای <code>[0.9995, 0.0005]</code> را پیش‌بینی کرده است. این‌ها <b>امتیازهایی هستند که به‌عنوان احتمال قابل‌تشخیص</b> شناخته می‌شوند.
</p>

<p>
برای به‌دست آوردن <b>برچسب‌هایی که با هر موقعیت متناظر هستند</b>، می‌توانیم ویژگی <code>id2label</code> را در پیکربندی مدل بررسی کنیم (توضیحات بیشتر در بخش بعدی خواهد آمد).
</p>

</div>


In [8]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}